## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-21-46-12 +0000,nypost,30K runners retrace fallen 9/11 hero’s path in...,https://nypost.com/2025/09/28/us-news/30k-runn...
1,2025-09-28-21-45-26 +0000,nyt,"Venezuela, America and the Specter of Regime C...",https://www.nytimes.com/2025/09/28/world/venez...
2,2025-09-28-21-43-10 +0000,bbc,Pro-EU party in Moldova takes lead in election...,https://www.bbc.com/news/articles/cx2rdlj8ejgo...
3,2025-09-28-21-39-00 +0000,wsj,Two Killed After Man Rams Into Michigan Church...,https://www.wsj.com/us-news/michigan-church-sh...
4,2025-09-28-21-38-19 +0000,nyt,The Michigan Attack Targeted a Latter-day Sain...,https://www.nytimes.com/2025/09/28/us/latter-d...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,24
34,church,12
103,shooting,11
61,new,11
57,eric,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,76
70,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...,70
7,2025-09-28-21-34-00 +0000,wsj,Opinion | Eric Adams Drops Out of the New York...,https://www.wsj.com/opinion/eric-adams-drops-o...,59
13,2025-09-28-21-27-11 +0000,bbc,At least two dead and several injured in shoot...,https://www.bbc.com/news/articles/ceq2vd15glwo...,59
15,2025-09-28-21-22-46 +0000,wapo,"At least two dead, eight injured in Michigan c...",https://www.washingtonpost.com/nation/2025/09/...,57


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,76,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
70,70,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...
13,59,2025-09-28-21-27-11 +0000,bbc,At least two dead and several injured in shoot...,https://www.bbc.com/news/articles/ceq2vd15glwo...
31,36,2025-09-28-20-41-51 +0000,bbc,Eric Adams ends re-election bid for New York C...,https://www.bbc.com/news/articles/cn761nkz6l5o...
14,28,2025-09-28-21-27-09 +0000,nypost,Hero cops responded to Grand Blanc LDS church ...,https://nypost.com/2025/09/28/us-news/hero-cop...
123,28,2025-09-28-12-12-08 +0000,cbc,"Russia pounds Kyiv, other regions in mass dron...",https://www.cbc.ca/news/world/russia-mass-dron...
166,26,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
28,22,2025-09-28-20-43-19 +0000,nypost,US Mission to UN warns NYPD to curb protesters...,https://nypost.com/2025/09/28/us-news/us-missi...
89,21,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...
0,20,2025-09-28-21-46-12 +0000,nypost,30K runners retrace fallen 9/11 hero’s path in...,https://nypost.com/2025/09/28/us-news/30k-runn...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
